In [ ]:
import os
import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import cortex
import pickle
from scipy.sparse import csr_matrix

def vol2surf(subject, stat):
    vol_data = cortex.Volume(stat.transpose(2,1,0), subject, 'NatPAC')
    mapper = cortex.get_mapper(subject, 'NatPAC', 'nearest')
    vertex_map = mapper(vol_data)

    mats = cortex.db.get_mri_surf2surf_matrix(subject=subject,
                                        surface_type='inflated',
                                        target_subj='fsaverage')

    left_transform = csr_matrix(mats[0])
    right_transform = csr_matrix(mats[1])
    vertex_map_fs_left = left_transform.dot(vertex_map.left)
    vertex_map_fs_right = right_transform.dot(vertex_map.right)
    vertex_map_fs = np.concatenate([vertex_map_fs_left, vertex_map_fs_right])
    return vertex_map_fs, vertex_map_fs_left, vertex_map_fs_right


In [ ]:


from scipy.stats import circmean
import cortex

# plotting group-averaged subject in fsaverage
subjects = ['sub-003', 'sub-004', 'sub-005','sub-006', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-015',
             'sub-016', 'sub-018', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-PBJ']

cmaps = ['angle_lh', 'angle_rh']
all_data = {}
gaze_types = ['mocet', 'polynomial', 'linear', 'uncorrected']
for gaze_type in gaze_types:
    angle_maps = []
    for subject in subjects:
        angle_map = nib.load(f'../../data/visual_field_mapping/freeviewing_results/'
                             f'{subject}_{gaze_type}_ang.nii').get_fdata()
        angle_map_surf, _, _ = vol2surf(subject, angle_map)
        angle_maps.append(angle_map_surf)
    angle_maps = circmean(angle_maps, axis=0, high=360)

    angle_maps[angle_maps == 0.0] = np.nan
    alpha_map_for_pRF = np.isnan(angle_maps)
    alpha_maps = np.zeros(angle_maps.shape)
    alpha_maps[angle_maps != np.nan] = 1

    for cmap in cmaps:
        vertex_data = cortex.Vertex(angle_maps, 'fsaverage', cmap=cmap, vmin=0, vmax=360)
        vertex_data = vertex_data.blend_curvature(alpha_maps, brightness=0.00, contrast=0.001, smooth=20)
        cortex.quickshow(vertex_data, with_colorbar=False, with_curvature=True,
                 with_rois=True, roi_list=['V1', 'V2', 'V3'],
                 linecolor=(1.0, 1.0, 1.0, 1.0), linewidth=1.5)
        all_data[f'{gaze_type}_{cmap}'] = vertex_data

angle_maps = []
for subject in subjects:
    angle_map = nib.load(f'../../data/visual_field_mapping/pRF_results/'
                         f'{subject}_params_ang.nii').get_fdata()
    angle_map_surf, _, _ = vol2surf(subject, angle_map)
    angle_maps.append(angle_map_surf)
angle_maps = circmean(angle_maps, axis=0, high=360)
angle_maps[alpha_map_for_pRF] = np.nan
for cmap in cmaps:
    vertex_data = cortex.Vertex(angle_maps, 'fsaverage', cmap=cmap, vmin=0, vmax=360)
    vertex_data = vertex_data.blend_curvature(alpha_maps, brightness=0.00, contrast=0.001, smooth=20)
    cortex.quickshow(vertex_data, with_colorbar=False, with_curvature=True,
             with_rois=True, roi_list=['V1', 'V2', 'V3'],
             linecolor=(1.0, 1.0, 1.0, 1.0), linewidth=1.5)
    plt.show()
    all_data['pRF' + '_' + cmap] = vertex_data
plt.show()

In [ ]:
# Azimuth 305, Altitude: 93, Radius 10, unfold 0, pivot 159, shift 90, depth 0.55
cortex.webshow(all_data)

In [ ]:
cmaps = ['magma_r']
gaze_types = ['mocet', 'polynomial', 'linear', 'uncorrected']
for gaze_type in gaze_types:
    eccentricity_maps = []
    for subject in subjects:
        eccentricity_map = nib.load(f'../../data/visual_field_mapping/freeviewing_results/'
                             f'{subject}_{gaze_type}_ecc.nii').get_fdata()
        eccentricity_map[eccentricity_map==0.0] = 99
        eccentricity_map_surf, _, _ = vol2surf(subject, eccentricity_map)
        eccentricity_maps.append(eccentricity_map_surf)
    eccentricity_maps = np.mean(eccentricity_maps, axis=0)
    if gaze_type == 'mocet':
        eccentricity_maps[np.logical_or(eccentricity_maps >= 90, eccentricity_maps == 0.0)] = np.nan
        alpha_map_for_pRF = np.isnan(eccentricity_maps)
        alpha_maps = np.zeros(eccentricity_maps.shape)
        alpha_maps[eccentricity_maps != np.nan] = 1
    else:
        eccentricity_maps[alpha_map_for_pRF] = np.nan

    for cmap in cmaps:
        vertex_data = cortex.Vertex(eccentricity_maps, 'fsaverage', cmap=cmap, vmin=0, vmax=20)
        vertex_data = vertex_data.blend_curvature(alpha_maps, brightness=0.00, contrast=0.001, smooth=20)
        cortex.quickshow(vertex_data, with_colorbar=False, with_curvature=True,
                 with_rois=True, roi_list=['V1', 'V2', 'V3'],
                 linecolor=(1.0, 1.0, 1.0, 1.0), linewidth=1.5)

eccentricity_maps = []
for subject in subjects:
    eccentricity_map = nib.load(f'../../data/visual_field_mapping/pRF_results/'
                         f'{subject}_params_ecc.nii').get_fdata()
    eccentricity_map_surf, _, _ = vol2surf(subject, eccentricity_map)
    eccentricity_maps.append(eccentricity_map_surf)
eccentricity_maps = np.mean(eccentricity_maps, axis=0)
eccentricity_maps[alpha_map_for_pRF] = np.nan
for cmap in cmaps:
    vertex_data = cortex.Vertex(eccentricity_maps, 'fsaverage', cmap=cmap, vmin=0, vmax=20)
    vertex_data = vertex_data.blend_curvature(alpha_maps, brightness=0.00, contrast=0.001, smooth=20)
    cortex.quickshow(vertex_data, with_colorbar=False, with_curvature=True,
             with_rois=True, roi_list=['V1', 'V2', 'V3'],
             linecolor=(1.0, 1.0, 1.0, 1.0), linewidth=1.5)
plt.show()